Simple inital EDA

TODO: these should be reusable methods calling statsbomb/whatever else api

In [ ]:
import json
import os

# Initialize an empty list to store all data
all_data = []

# Path to your json files
json_folder_path = '../data/raw_matches'

# Loop over all json files
for file in os.listdir(json_folder_path):
    if file.endswith(".json"):
        with open(os.path.join(json_folder_path, file), 'r') as f:
            # Load data from json file and append to all_data list
            data = json.load(f)
            all_data.extend(data)

# Write combined data to new json file
with open('combined_data.json', 'w') as f:
    json.dump(all_data, f)


In [ ]:
import json
import os
from tqdm import tqdm
import pandas as pd

# Path to your json files
json_folder_path = '../data/events'

with open(os.path.join('../data', "combined_matches.json"), 'r') as f:
    matches = json.load(f)
    match_id_list = [str(match['match_id']) for match in matches]

# Loop over all json files
shot_data = []
for file in tqdm(os.listdir(json_folder_path)):
    if file.split('.')[0] in match_id_list:
        with open(os.path.join(json_folder_path, file), 'r') as f:
            # Load data from json file and append to all_data list
            match = json.load(f)
            for event in match:
                if event['type']['name'].lower() == "shot".lower():
                    for key, val in event["shot"].items():
                        event[key] = val
                    del event["shot"]
                    shot_data.append(event)

df = pd.DataFrame(shot_data)
df.to_csv('shot_data.csv')

In [ ]:
# Some tidy up

df["play_pattern"] = df["play_pattern"].apply(lambda x: x["name"])
df["position"] = df["position"].apply(lambda x: x["name"])
df["x_location"] = df["location"].apply(lambda x: x[0])
df["y_location"] = df["location"].apply(lambda x: x[1])
df["technique"] = df["technique"].apply(lambda x: x["name"])
df["body_part"] = df["body_part"].apply(lambda x: x["name"])

df.to_csv('shot_data_clean.csv')

In [ ]:
df["outcome"] = df["outcome"].apply(lambda x: x["name"])

df.to_csv('shot_data_clean.csv')

In [ ]:
# Load the data from the csv file again
df = pd.read_csv("/mnt/data/shot_data.csv")

# Parse the 'shot' column
df['shot'] = df['shot'].apply(parse_dict)

# Extract the shot outcome
df['shot_outcome_name'] = df['shot'].apply(lambda x: x['outcome']['name'] if 'outcome' in x else np.nan)

# Create a binary 'goal' variable
df['goal'] = df['shot_outcome_name'].apply(lambda x: 1 if x == 'Goal' else 0)

# Display the first few rows of the updated DataFrame
df.head()